In [2]:
# attacker has PU and ciphertext
# generate 2^56 keys
# send one key as message and E with PU
# compare ciphers

In [45]:
# !pip install pycrypto

In [1]:
def get_keys():
    keys= []
    for i in range(0,pow(2,27)):   # 134,217,728 keys
        i = hex(i)
        val = i[2:]
        
        for i in range(16):
            if len(val) < 16:
                val = '0' + val
        
        keys.append(val)
        
    return keys

In [2]:
# %time k = get_keys()

Wall time: 12min 5s


In [11]:
# selecting M
k[72663296]

'000000000454c100'

In [12]:
import random


'''
Euclid's algorithm for determining the greatest common divisor
Use iteration to make it faster for larger integers
'''

def gcd(a, b):
    while b != 0:
        a, b = b, a % b
    return a


'''
Euclid's extended algorithm for finding the multiplicative inverse of two numbers
'''


def multiplicative_inverse(e, phi):
    d = 0
    x1 = 0
    x2 = 1
    y1 = 1
    temp_phi = phi

    while e > 0:
        temp1 = temp_phi//e
        temp2 = temp_phi - temp1 * e
        temp_phi = e
        e = temp2

        x = x2 - temp1 * x1
        y = d - temp1 * y1

        x2 = x1
        x1 = x
        d = y1
        y1 = y

    if temp_phi == 1:
        return d + phi


'''
Tests to see if a number is prime.
'''


def is_prime(num):
    if num == 2:
        return True
    if num < 2 or num % 2 == 0:
        return False
    for n in range(3, int(num**0.5)+2, 2):
        if num % n == 0:
            return False
    return True


def generate_key_pair(p, q):
    if not (is_prime(p) and is_prime(q)):
        raise ValueError('Both numbers must be prime.')
    elif p == q:
        raise ValueError('p and q cannot be equal')
    # n = pq
    n = p * q

    # Phi is the totient of n
    phi = (p-1) * (q-1)

    # Choose an integer e such that e and phi(n) are coprime
    e = random.randrange(1, phi)

    # Use Euclid's Algorithm to verify that e and phi(n) are coprime
    g = gcd(e, phi)
    while g != 1:
        e = random.randrange(1, phi)
        g = gcd(e, phi)

    # Use Extended Euclid's Algorithm to generate the private key
    d = multiplicative_inverse(e, phi)

    # Return public and private key_pair
    # Public key is (e, n) and private key is (d, n)
    return ((e, n), (d, n))


def encrypt(pk, plaintext):
    # Unpack the key into it's components
    key, n = pk
    
    # Convert each letter in the plaintext to numbers based on the character using a^b mod m
    cipher = [pow(ord(char), key, n) for char in plaintext]
#     cipher = cipher[1:-1]
    
    string=''
    for i in cipher:
        string = string+str(i)
                
    # Return the array of bytes
    return int(string), cipher


def decrypt(pk, ciphertext):
    # Unpack the key into its components
    key, n = pk
    # Generate the plaintext based on the ciphertext and key using a^b mod m
    aux = [str(pow(char, key, n)) for char in ciphertext]
    # Return the array of bytes as a string
    plain = [chr(int(char2)) for char2 in aux]
    return ''.join(plain)


if __name__ == '__main__':
    '''
    Detect if the script is being run directly by the user
    '''
    print("===========================================================================================================")
    print("================================== RSA Encryptor / Decrypter ==============================================")
    print(" ")

    p = int(input(" - Enter a prime number (17, 19, 23, etc): "))
    q = int(input(" - Enter another prime number (Not one you entered above): "))

    print(" - Generating your public / private key-pairs now . . .")

    public, private = generate_key_pair(p, q)

    print(" - Your public key is ", public, " and your private key is ", private)

    message = input(" - Enter a message to encrypt with your public key: ")
    encrypted_msg, for_decryption = encrypt(public, message)

    print(" - Your encrypted message is: ", encrypted_msg)
    print(" - Decrypting message with private key ", private, " . . .")
    print(" - Your message is: ", decrypt(private, for_decryption))

    print(" ")
    print("============================================ END ==========================================================")
    print("===========================================================================================================")

================================== RSA Encryptor / Decrypter ==============================================
 
 - Enter a prime number (17, 19, 23, etc): 73
 - Enter another prime number (Not one you entered above): 89
 - Generating your public / private key-pairs now . . .
 - Your public key is  (4739, 6497)  and your private key is  (5483, 6497)
 - Enter a message to encrypt with your public key: 000000000454c100
 - Your encrypted message is:  3846384638463846384638463846384638462124639821241864562438463846
 - Decrypting message with private key  (5483, 6497)  . . .
 - Your message is:  000000000454c100
 
============================================ END ==========================================================


In [8]:
def prob_msg_attack(pubkey, ciphertext):
    for i in k:
        c, x = encrypt(pubkey,i)
        if c == ciphertext:
            break
    return i

In [13]:
%time M = prob_msg_attack((4739,6497),3846384638463846384638463846384638462124639821241864562438463846)
print(M)

Wall time: 1h 7min 10s
000000000454c100
